This notebook contain the steps to go from nedcow postgresql database to csv file.

In [1]:
import psycopg2
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
%matplotlib inline

In [2]:
db_string = "postgresql:///nedcow-dev"

db = create_engine(db_string)

db = psycopg2.connect("dbname='nedcow-dev' host='localhost' password='mafia=612'")

cur = db.cursor()

cur.execute("Select * from results")

rows = cur.fetchall()

Data(rows) is returned as a list of tuples
The list of tuples contain
    index 0: Database index ID - type - int
    index 1: Store Name - type - str
    index 2: Product Information - type - Dict
    index 3: Amazon ASIN id - type - str
    index 4: Date - type - datetime object

In [3]:
rows[0]

(1,
 'target',
 {'amazon_sales_price': '75.98',
  'asin': 'B004QNL2CC',
  'image': 'http://ecx.images-amazon.com/images/I/51LTTrOVc9L._SL75_.jpg',
  'margin': '47.70',
  'pagelink': 'http://m.target.com/p/our-generation-20-morgan-horse-with-accessories/-/A-13407851',
  'profit': '16.69',
  'purchaseprice': '34.99',
  'salesrank': '17704',
  'title': 'Our Generation Poseable Morgan Horse For 18" Dolls',
  'upc': '062243242999'},
 'B004QNL2CC',
 datetime.datetime(2014, 6, 8, 19, 35, 45, 958895))

#### Make list of dictionaries to load into pandas Dataframe

In [4]:
# There has to be a more elegant way to do this
new_dict = {}
new_list = []
for row in rows:
    new_dict = row[2]
    new_dict['store'] = row[1]
    new_dict['date'] = row[4]
    new_list.append(new_dict)
 

In [5]:
df = pd.DataFrame(new_list)

In [6]:
df.shape

(263181, 18)

In [8]:
df = df[df['purchaseprice'] != '']

In [9]:
df.shape

(263177, 18)

In [10]:
df.dtypes

amazon_sales_price            object
asin                          object
date                  datetime64[ns]
image                         object
logo                          object
margin                        object
model                         object
model#                        object
pagelink                      object
profit                        object
purchaseprice                 object
salesrank                     object
shipping                      object
sku                           object
store                         object
storepick                     object
title                         object
upc                           object
dtype: object

#### Set columns to appropriate datatype
I am trying to convert string values that should be integer or floats

If a true string value exists within the values then I check the string size.

If the string size is greater than 10 characters then I set the value to 0.0
This done with the 'check_len' function.

In [11]:
def check_len(x):
    if isinstance(x, str):
        x = x.strip().replace(',','')
        if len(x) > 20:
            x = '0'
            return x
        else:
            return x
    else:
        x = str(x)
        return x

In [12]:
df['amazon_sales_price'] = df.amazon_sales_price.replace(',','').astype(float)
df['asin'] = df.asin.astype(str)
df['image'] = df.image.astype(str)
df['logo'] = df.logo.astype(str)
df['margin'] = df.margin.replace(',','').astype(float)
df['model'] = df.model.astype(str)
df['model#'] = df['model#'].astype(str)
df['pagelink'] = df.pagelink.astype(str)
df['profit'] = df.profit.replace(',','').astype(float)
df['purchaseprice'] = df.purchaseprice.apply(lambda x: check_len(x)).astype(float)
df['salesrank']=df.salesrank.astype(int)
df['shipping'] = df.shipping.astype(str)
df['sku'] = df.sku.astype(str)
df['store'] = df.store.astype(str)
df['storepick'] = df.storepick.astype(str)
df['title'] = df.title.astype(str)
df['upc'] = df.upc.astype(str)


In [13]:
df = df[df['purchaseprice'] > 0]

In [14]:
df.to_csv('goodstuff.csv.xz', compression='xz', index=False)

In [15]:
print(df.shape)
print(df.head())

(256955, 18)
   amazon_sales_price        asin                       date  \
0               75.98  B004QNL2CC 2014-06-08 19:35:45.958895   
1               21.99  B007GQOCUK 2014-06-08 19:35:45.958895   
2               36.95  B0073FV4CS 2014-06-08 19:35:45.958895   
3               29.95  B00AHVJ9V8 2014-06-08 19:35:45.958895   
4               69.95  B006O3F3KO 2014-06-08 19:35:45.958895   

                                               image logo  margin model  \
0  http://ecx.images-amazon.com/images/I/51LTTrOV...  nan   47.70   nan   
1  http://ecx.images-amazon.com/images/I/61zYcBOF...  nan   43.28   nan   
2  http://ecx.images-amazon.com/images/I/41uWeEHI...  nan   63.68   nan   
3  http://ecx.images-amazon.com/images/I/41TGh7cq...  nan   40.29   nan   
4  http://ecx.images-amazon.com/images/I/41Dkj6oZ...  nan   83.81   nan   

  model#                                           pagelink  profit  \
0    nan  http://m.target.com/p/our-generation-20-morgan...   16.69   
1    nan 